In [1]:
import spacy
from spacy.pipeline import EntityRuler
from spacy.lang.en import English
from spacy.tokens import Doc

In [2]:
import gensim
from gensim import corpora

In [3]:
from spacy import displacy
import numpy as np 
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import jsonlines
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download(['stopwords', 'wordnet'])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shant\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shant\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
import warnings 
warnings.filterwarnings('ignore')

In [6]:
df = pd.read_csv('Resume.csv')
df=df.reindex(np.random.permutation(df.index))


In [10]:
data = df.copy().iloc[
    0:200,
]
data.head()


,ID,Resume_str,Resume_html,Category
1549,81217013,"DIRECTOR, FINANCE OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",FINANCE
426,58105060,MATH TEACHER Menrit Mansour...,"<div class=""fontsize fontface vmargins hmargin...",TEACHER
397,19302310,DIRECTOR/PRESCHOOL TEACHER Summ...,"<div class=""fontsize fontface vmargins hmargin...",TEACHER
1433,38309905,EXECUTIVE CHEF III Summary ...,"<div class=""fontsize fontface vmargins hmargin...",CHEF
1218,21156767,Pavithra Shetty Summary ...,"<div class=""RNA skn-mlf9 fontsize fontface vma...",CONSULTANT


In [12]:
data.info()
data.value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 1549 to 2349
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           200 non-null    int64 
 1   Resume_str   200 non-null    object
 2   Resume_html  200 non-null    object
 3   Category     200 non-null    object
dtypes: int64(1), object(3)
memory usage: 7.8+ KB


ID        Resume_str                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [14]:
nlp = spacy.load("en_core_web_lg")
skill_pattern_path = "jz_skill_patterns.jsonl"

In [15]:
ruler=nlp.add_pipe("entity_ruler")
ruler.from_disk(skill_pattern_path)
nlp.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'entity_ruler']

In [19]:
#Extract the skills
def get_skills(text):
    doc = nlp(text)
    myset = []
    subset = []
    for ent in doc.ents:
        if ent.label_ == 'SKILL':
            subset.append(ent.text)
    myset.append(subset)
    return subset
def unique_skills(x):
    return list(set(x)) 

In [17]:
#Text cleaning
clean = []
for i in range(data.shape[0]):
    review = re.sub(
        '(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"',
        " ",
        data["Resume_str"].iloc[i],
    )
    review = review.lower()
    review = review.split()
    lm = WordNetLemmatizer()
    review = [
        lm.lemmatize(word)
        for word in review
        if not word in set(stopwords.words("english"))
    ]
    review = " ".join(review)
    clean.append(review)

In [20]:
data["Clean_Resume"]= clean
data["skills"]=data["Clean_Resume"].str.lower().apply(get_skills)
data["skills"]=data["skills"].apply(unique_skills)
data.head()

,ID,Resume_str,Resume_html,Category,Clean_Resume,skills
1549,81217013,"DIRECTOR, FINANCE OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",FINANCE,director finance operation executive profile m...,"[schedule, accounting, project management, bus..."
426,58105060,MATH TEACHER Menrit Mansour...,"<div class=""fontsize fontface vmargins hmargin...",TEACHER,math teacher menrit mansour professional summa...,"[schedule, testing, comprehension, medium, col..."
397,19302310,DIRECTOR/PRESCHOOL TEACHER Summ...,"<div class=""fontsize fontface vmargins hmargin...",TEACHER,director preschool teacher summary personable ...,"[interaction, documentation, schedule]"
1433,38309905,EXECUTIVE CHEF III Summary ...,"<div class=""fontsize fontface vmargins hmargin...",CHEF,executive chef iii summary experienced well ro...,"[schedule, chef]"
1218,21156767,Pavithra Shetty Summary ...,"<div class=""RNA skn-mlf9 fontsize fontface vma...",CONSULTANT,pavithra shetty summary customer oriented prin...,"[data warehouse, documentation, correctness, s..."


In [23]:
import plotly.express as px
fig = px.histogram(
    data, x="Category", title="Distribution of Jobs Categories"
).update_xaxes(categoryorder="total descending")
fig.show()

In [24]:
Job_cat = data["Category"].unique()
Job_cat = np.append(Job_cat, "ALL")

In [26]:
sent = nlp(data["Resume_str"].iloc[0])
displacy.render(sent, style="ent", jupyter=True)

In [27]:
displacy.render(sent[0:10], style="dep", jupyter=True, options={"distance": 90})

In [29]:
patterns = df.Category.unique()
for a in patterns:
    ruler.add_patterns([{"label": "Job-Category", "pattern": a}])

In [30]:
colors = {
    "Job-Category": "linear-gradient(90deg, #aa9cfc, #fc9ce7)",
    "SKILL": "linear-gradient(90deg, #9BE15D, #00E3AE)",
    "ORG": "#ffd966",
    "PERSON": "#e06666",
    "GPE": "#9fc5e8",
    "DATE": "#c27ba0",
    "ORDINAL": "#674ea7",
    "PRODUCT": "#f9cb9c",
}
options = {
    "ents": [
        "Job-Category",
        "SKILL",
        "ORG",
        "PERSON",
        "GPE",
        "DATE",
        "ORDINAL",
        "PRODUCT",
    ],
    "colors": colors,
}
sent = nlp(data["Resume_str"].iloc[5])
displacy.render(sent, style="ent", jupyter=True, options=options)

In [31]:
input_resume = "Shantanu GuptaAspiring Software DeveloperRaipur, Chhattisgarh+91 9752005866shantanugupta0412@gmail.comEDUCATIONIIIT Naya Raipur — B.Tech in Computer Science andEngineeringNovember 2022 - PresentCGPA: 8.35Swami Vivekananda Senior Secondary School, Raipur —12th CBSE2022Grade: 92.4%Bharatiya Vidya Bhavans R.K. Sarda Vidya Mandir, Raipur— 10th CBSE2020Grade: 95%PROJECTSSocial Media Sentiment Analysis — ML and Web DevelopmentJuly 2023 - December 2023-Designed and Built an ML model for sentiment analysis of Twitter, YouTube andAmazon reviews.-Developed a full stack web application on React and Flask for manual testing ofthe model.Fake News Detection using Machine Learning — MLMarch 2023 - June 2023-Built an NLP based ML model which detects fake news from news articles.-Implemented it on a Tkinter based user interface.Smart Cradle and Baby Monitoring System — Raspberry Pi andPythonMarch 2023 - June 2023Built a baby cradle with multiple sensors including temperature, humidity andaccelerometer. A camera was trained using a CV algorithm for detection of the babyas well as a mailing system was developed to notify of any discrepancies detected.Raspberry Pi acted as a controlling hub and the programs were coded in Python.SKILLSProficient With:C++, JavaScript, React, HTML,CSSFamiliar With:Node.js, Python(NumPy andPandas), Flask, SQL, MachineLearning, Raspberry Pi,ArduinoCERTIFICATIONSCisco:● JavaScript Essentials 1● JavaScript Essentials 2Forage:● J.P. Morgan ChaseSoftware EngineeringVirtual ExperienceLANGUAGESWorking Proficiency:EnglishNative Proficiency:HindiHOBBIES● Reading● Cinema● SpeedCubing"
sent2 = nlp(input_resume)
displacy.render(sent2, style="ent", jupyter=True, options=options)

In [34]:
#Match score
input_skills = "C++,Python,JavaScript,SQL,Machine learning"
req_skills = input_skills.lower().split(",")
resume_skills = unique_skills(get_skills(input_resume.lower()))
score = 0
for x in req_skills:
    if x in resume_skills:
        score += 1
req_skills_len = len(req_skills)
match = round(score / req_skills_len * 100, 1)

print(f"The current Resume is {match}% matched to your requirements")

The current Resume is 40.0% matched to your requirements
